<br>

# Introdução

In [ ]:
import os
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

<br>

# Get List Links

In [ ]:
# Get Data
url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
soup = soup.find('select',  attrs={'id': 'local_mun'})
rows = soup.find_all('option')

In [ ]:
# Listas Vazias
list_url = []
list_name = []

# Append as Listas
for row in rows:    
    list_name.append(row.get_text())
    list_url.append(row['value'])

# Lista de Tuplas
list_tuples = list(zip(list_name, list_url))[1:]

In [ ]:
# Convert to Dataframe
df = pd.DataFrame(list_tuples, columns = ['municipio', 'url'])
df['url'] = f"{url}{df['url']}/"

# Fixed variable
list_df_urls = df.iterrows()

# Results
df.head()

<br>

# Get Data for Município

<br>

## Function

In [ ]:
def get_data(municipio, url):
    # Get Data
    #url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/motuca'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    soup = soup.find('form',  attrs={'name': 'form2'}).find_next_sibling('div')

    # List
    list_data = soup.find('p').get_text().split('\n')
    dict_data = {
        'municipio_nome': municipio,
        'ida': soup.find('h2').text,
        'endereco': ([x for x in list_data if x.startswith('Endereço:')] or [''])[0],
        'municipio_sede': ([x for x in list_data if x.startswith('Município:')] or [''])[0],
        'telefone': ([x for x in list_data if x.startswith('Telefone:')] or [''])[0],
        'fax': ([x for x in list_data if x.startswith('Fax:')] or [''])[0],
        'email': ([x for x in list_data if x.startswith('E-mail:')] or [''])[0],
        'unidaderegional': ([x for x in list_data if x.startswith('Unidade Regional:')] or [''])[0],
        'diretor': ([x for x in list_data if x.startswith('Diretor:')] or [''])[0].replace('\r', ''),
        'url': url,
    }
    return dict_data

In [ ]:
url = 'https://www.defesa.agricultura.sp.gov.br/enderecos/pedra-bela/'
get_data('Mombuca', url)

<br>

## Loop

In [ ]:
list_dicts = []
for index, row in list_df_urls:
    # Get values
    municipio = row['municipio']
    url = row['url']
    print(f'>>>>> {url}')
    print(f'{municipio} > {url}')

    # Append
    list_dicts.append(get_data(municipio, url))

<br>

# Create Dataframe

In [ ]:
# Create dataframe
df = pd.DataFrame(list_dicts)

In [ ]:
# Divide Células em Duas Colunas
df[['endereco','cep']] = df['endereco'].str.split('- CEP:', expand=True)
df[['municipio_sede','estado']] = df['municipio_sede'].str.split('/', expand=True)
df[['tel_1','tel_2']] = df['telefone'].str.split('/', expand=True)
df[['fax_1','fax_2']] = df['fax'].str.split('/', expand=True)

In [ ]:
# Remove tudo que vem antes do :
df = df.applymap(lambda x: x.split(':')[-1], na_action='ignore')

# Strip All Data
df = df.applymap(lambda x: x.strip(), na_action='ignore')

<br>

## Rename

In [ ]:
def rename_nome(x):
    #x = x.title()
    x = x.strip()
    dict_rename = {
        # Encoding
        '  '  : ' ',

        # Erros
        '()'  : '',
        'Av. '  : 'Avenida ',
        'R. '  : 'Rua ',
        
        # Abreviações
        'Mogi-Mirim' : 'Mogi Mirim',
        's/nº': 's/n',
        's/n°': 's/n',
        's/n': 's/n',
        'S/N' : 's/n',
        'S/n' : 's/n',
        's/n' : 's/n',
        
        ' nº ': ', ',
        ', nº ': ', ',
        ', ,': ',',
        ',,': ',',

        'EDA de ': 'Escritório de Defesa Agropecuária de ',

        # 
        'RogÉrio': 'Rogério',
        'JosÉ': 'José',
        'CÉsar': 'César',
        'MarÇal': 'Marçal',
        'AdÃo': 'Adão',
        'TristÃo': 'Tristão',
        'ConceiÇÃo': 'Conceição',
        
        'Avednida ':'Avenida ',
        'Av.25 de Janeiro':'Avenida 25 de Janeiro',
        'Rua 13 de maio, 260': 'Rua 13 de Maio, 260',
        ' Do ': ' do ',
        'RUA FUAD EID,319': 'Rua Fuad Eid, 319',

    }
    for k, v in dict_rename.items():
        x = x.replace(k, v)
    x = x.replace('  ', ' ')
    return x.strip()

In [ ]:
# Rename
df.loc[:, 'ida'] = df['ida'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'endereco'] = df['endereco'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'municipio_sede'] = df['municipio_sede'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'tel_1'] = df['tel_1'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'unidaderegional'] = df['unidaderegional'].astype(str).apply(lambda x: rename_nome(x))
df.loc[:, 'diretor'] = df['diretor'].astype(str).apply(lambda x: rename_nome(x))

In [ ]:
# Fix Municípios
df['municipio_nome'] = df['municipio_nome'].replace('Embu', 'Embu das Artes').replace('Mogi-Mirim', 'Mogi Mirim')

# Fix Telefone
df['tel_1'] = df['tel_1'].replace('(11)', '').replace('(18)', '')

<br>

## Ajustes

In [ ]:
# Adjust Error
df.set_index('municipio_nome', inplace=True)
df.loc['Pontes Gestal', 'ida'] = 'Unidade de Defesa Agropecuária de Pontes Gestal'
df.reset_index(inplace=True)

In [ ]:
# Preenche espaços em branco por NaN
df.loc[:, 'unidaderegional'] = df['unidaderegional'].replace(r'^\s*$', np.nan, regex=True)

# Define máscara
mask = df['unidaderegional'].isnull()

# Para os que estão em branco, define que é igual ao ida
df.loc[mask, 'unidaderegional'] = df['ida']

<br>

## Join Municípios

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df_mun = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/tabs/tab_municipio_nome.csv',
    usecols=['id_municipio', 'municipio_nome']
)

# Merge
df = pd.merge(
    df_mun,
    df,
    how='left',
    left_on='municipio_nome',
    right_on='municipio_nome'
)

# Resultados
df.head()

<br>

## Ordena Colunas

In [ ]:
print(list(df.columns))

df = df[[
    'id_municipio',
    'municipio_nome',
    'ida',
    'endereco',
    'cep',
    'municipio_sede',
    'tel_1',
    'tel_2',
    'fax_1',
    'fax_2',
    'email',
    'unidaderegional',
    'diretor',
    'url',
]]

<br>

# Salva

In [ ]:
from paths import *

In [ ]:
# Results
df.to_csv(
    os.path.join(output_path_tabs, 'tab_municipio_eda.csv'),
    index=False
)